# Installing libraries

In [1]:
import lap
import math
import sys
import util

import numpy as np
import scipy as sp

from lap import lapmod   
from lap import lapjv   

## Getting coordinates present in frame 

In [2]:
def get_coordinates(coords, frame):
    # Identifying rows of "coords" with current frame number
    frame_coords = coords[coords['T'] == frame]

    return frame_coords

## Initialise first timepoint
All cells detected in the first timepoint can be considered "new" tracks.  As such, we give them each a unique track ID number.

In [3]:
def initialise_first_timepoint(coords):
    # Adding a blank extra column for the track ID
    coords['TRACK_ID'] = 0

    # Getting the row indices for the first frame
    idx_first = coords['T'] == 0

    coords.loc[idx_first,'TRACK_ID'] = range(1,sum(idx_first)+1)

# Calculating linking costs

In [4]:
## Arguments
# prev = "coords" table for points in previous frame
# curr = "coords" table for points in current frame
# thresh = maximum linking distance for points to move between two frames

## Return
# n_rows = the number of rows in the cost matrix
# cc = array starting at [0,0] and proceeding row-by-row, these are the cost values
# ii = array holding cumulative number of non-zero costs after each row.  Has an extra 0 at the start
# kk = cost matrix column index of each element in cc
# prev_rows = row indices (for coords array) for points in previous frame
# curr_rows = row indices (for coords array) for points in current frame
# transposed = boolean denoting if cost matrix is transposed

def calculate_sparse_cost_matrix(prev,curr,thresh):
#     row_ind = []
#     col_ind = []
#     data = []
#     for prev_idx,(prev_row,prev_pt) in enumerate(prev.iterrows()):
#         for curr_idx,(curr_row,curr_pt) in enumerate(curr.iterrows()):
#             # Calculating cost
#             cost = calculate_cost(prev_pt,curr_pt,thresh)
            
#             # Only include costs lower than the threshold
#             if cost <= thresh:
#                 row_ind.append(prev_idx)
#                 col_ind.append(curr_idx)
#                 data.append(cost)
    
#     spa = sp.sparse.csc_matrix((data,(row_ind,col_ind)))
#     print(spa)
                
    # Initalising variables
    n_NZ = 0
    cc = []
    ii = [0] # ii has an extra 0 at the start
    kk = []
            
    # Iterating over each pair, calculating the cost
    for prev_idx,(prev_row,prev_pt) in enumerate(prev.iterrows()):
        for curr_idx,(curr_row,curr_pt) in enumerate(curr.iterrows()):
            # Calculating cost
            cost = calculate_cost(prev_pt,curr_pt,thresh)
            
            # Only include costs lower than the threshold
            if cost <= thresh:
                cc.append(cost)
                kk.append(curr_idx)
                n_NZ = n_NZ + 1
                
        # Adding the current number of costs
        ii.append(n_NZ)
        
    # Creating lists of object indices within main coordinates table
    prev_rows = [prev_row for (prev_row,_) in prev.iterrows()]
    curr_rows = [curr_row for (curr_row,_) in curr.iterrows()]
              
    # Removing columns without values.  Iterate over each possible value, checking if there's an instance of it
    kk_np = np.array(kk)
    for idx in range(len(curr_rows)-1,-1,-1):
        # If this value isn't present, shift all remaining columns down by 1
        if np.sum(kk_np==idx) == 0:
            del curr_rows[idx]
                
    for idx in range(0,kk_np.max()+1):
        # If this value isn't present, shift all remaining columns down by 1
        if np.sum(kk_np==idx) == 0:
            kk_np[kk_np>idx] = kk_np[kk_np>idx]-1
                    
    # Remmoving empty rows.  Reverse iterate over all values in ii.  If a value is the same as the previous, remove this row
    for idx in range(len(ii)-1,0,-1):
        if(ii[idx] == ii[idx-1]):
            ii.remove(ii[idx-1])
            del prev_rows[idx-1]
            
    
    # Checking there are more rows than columns; if not, transposing the cost matrix
    if len(prev_rows) < len(curr_rows):
        n_rows, cc, ii, kk, prev_rows, curr_rows, transposed = calculate_sparse_cost_matrix(curr,prev,thresh)
        return n_rows, cc, ii, kk, prev_rows, curr_rows, True
    else:
        return len(ii)-1, np.array(cc), np.array(ii), kk_np, prev_rows, curr_rows, False
                
    
def calculate_cost(prev_pt,curr_pt,thresh):
    # Spatial linking (distance between two points)
    dx = (curr_pt.X-prev_pt.X)
    dy = (curr_pt.Y-prev_pt.Y)
    d = math.sqrt(dx*dx + dy*dy)
    
    # If the two points are separated by more than the linking threshold, set them to infinity ('inf')
    if d > thresh:
        d = math.inf
    
    return d

# Running an example for a pair of frames

In [5]:
# Setting parameters
np.set_printoptions(precision=3,threshold=sys.maxsize)
linking_thresh = 10

# Loading coordinates
path = "../data/TestObjectCoordinatesNoHeader.csv"
coords = util.load_coordinates(path);

# Set new track IDs for each object in the first frame
initialise_first_timepoint(coords)

frame = 1
# Get coordinates from the previous frame
prev_coords = get_coordinates(coords,frame-1)
    
# Get coordinates from the current frame
curr_coords = get_coordinates(coords,frame)
    
print("Prev:\n",prev_coords,"\n")    
print("Curr:\n",curr_coords,"\n")
    
# Calculate costs for each possible link    
n_rows, cc, ii, kk, prev_rows, curr_rows, transposed = calculate_sparse_cost_matrix(prev_coords,curr_coords,linking_thresh)
print("Sparse costs:\nn_rows = %i \ncc = %a \nii = %a \nkk = %a \nprev_rows = %a \ncurr_rows = %a \ntransposed = %s \n" 
          % (n_rows,cc,ii,kk,prev_rows,curr_rows,transposed))
    
# Calculate assigninments for each
opt,x,y = lapmod(n_rows,cc,ii,kk)
print("Total cost =",opt)
print("x = %a" % x)
print("y = %a" % y)

Loading coordinates from " ../data/TestObjectCoordinatesNoHeader.csv "
Loaded data shape:  (13, 6)
 
Prev:
      ID           X           Y    T         AREA  INTENSITY  TRACK_ID
0  44.0   25.000000  495.000000  0.0  34052.37681       69.0         1
1  45.0   44.000000  581.000000  0.0  33118.20725      193.0         2
2  46.0   55.960422  513.007916  0.0  33789.78100      379.0         3
3  50.0  166.000000  572.000000  0.0  33925.10492      305.0         4
4  51.0  168.000000   32.000000  0.0  33395.35556       45.0         5
5  49.0  118.000000  468.500000  0.0  33935.66915      402.0         6
6  52.0  182.000000  497.000000  0.0  33086.95172      145.0         7
7  54.0   25.000000  494.000000  0.0  34052.37681       69.0         8 

Curr:
      ID           X           Y    T         AREA  INTENSITY  TRACK_ID
8   1.0   24.000000  499.000000  1.0  34061.72464       69.0         0
9   2.0   44.000000  581.000000  1.0  33106.98446      193.0         0
10  3.0   51.209726  516.325228